# Getting Data for Sentiment Analysis - Tesla and Elon Musk

In [59]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()



True

In [60]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/georgecruz/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [61]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [62]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



# Fetching TESLA News usineg the API

In [63]:
# Fetch top news articles about Tesla
tesla_headlines = newsapi.get_everything(
    q="tesla", language="en", sort_by="relevancy"
)

# Print total articles
print(f"Total articles about TESLA: {tesla_headlines['totalResults']}")

# Show sample article
tesla_headlines["articles"][0]




Total articles about TESLA: 6147


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Jon Fingas',
 'title': 'Tesla Supercharger flaw lets rival EVs top up for free in Europe',
 'description': 'Tesla’s Supercharger stations normally aren’t very welcoming places to competing electric cars, but European chargers may be another story — for now. Electrek pointed to reports from Nextmove and others that Supercharger V3 stations in Europe are letting peop…',
 'url': 'https://www.engadget.com/tesla-v3-superchargers-allow-free-charges-for-rival-evs-194629138.html',
 'urlToImage': 'https://o.aolcdn.com/images/dims?resize=1200%2C630&crop=1200%2C630%2C0%2C0&quality=95&image_uri=https%3A%2F%2Fs.yimg.com%2Fos%2Fcreatr-uploaded-images%2F2020-09%2F7b545800-f52d-11ea-aff5-1509510515df&client=amp-blogside-v2&signature=9a92d113d6d38444ff68b683ad928ed77ab78eb7',
 'publishedAt': '2020-09-12T19:46:29Z',
 'content': 'We’ve asked Tesla for comment. It has previously welcomed the possibility of allowing Supercharger access to non-T

### There are more than 6000 top news related to TESLA. 

In [64]:
# Transform the response dictionary to a DataFrame
tesla_df = pd.DataFrame.from_dict(tesla_headlines["articles"])

tesla_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,Tesla Supercharger flaw lets rival EVs top up ...,Tesla’s Supercharger stations normally aren’t ...,https://www.engadget.com/tesla-v3-supercharger...,https://o.aolcdn.com/images/dims?resize=1200%2...,2020-09-12T19:46:29Z,We’ve asked Tesla for comment. It has previous...
1,"{'id': None, 'name': 'Lifehacker.com'}",David Murphy,How to Watch Tesla's 'Battery Day' Today,It’s Tesla day! Whether you think Elon Musk is...,https://lifehacker.com/how-to-watch-teslas-bat...,https://i.kinja-img.com/gawker-media/image/upl...,2020-09-22T19:00:00Z,Its Tesla day! Whether you think Elon Musk is ...
2,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,Why Amazon and Panasonic are betting on this b...,"JB Straubel, the Tesla co-founder and former C...",http://techcrunch.com/2020/10/09/why-amazon-an...,https://techcrunch.com/wp-content/uploads/2020...,2020-10-09T19:19:05Z,"JB Straubel, the Tesla co-founder and former C..."
3,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,How to watch Tesla’s double hitter shareholder...,Tesla’s battery day is here after months of tw...,http://techcrunch.com/2020/09/22/how-to-watch-...,https://techcrunch.com/wp-content/uploads/2020...,2020-09-22T14:57:14Z,Tesla’s battery day is here after months of tw...
4,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,Elon Musk reveals the Tesla Model S ‘Plaid’ sl...,Tesla CEO Elon Musk revealed Tuesday the Model...,http://techcrunch.com/2020/09/22/elon-musk-rev...,https://techcrunch.com/wp-content/uploads/2020...,2020-09-23T01:48:54Z,Tesla CEO Elon Musk revealed Tuesday the Model...


## Create the Sentiment Score Table - Tesla News

In [65]:
# Create the TESLA sentiment scores DataFrame
tesla_sentiments = []

for article in tesla_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tesla_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
tesla_df = pd.DataFrame(tesla_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
tesla_df = tesla_df[cols]

tesla_df.head()



,date,text,compound,positive,negative,neutral
0,2020-09-12,We’ve asked Tesla for comment. It has previous...,0.7543,0.220,0.067,0.713
1,2020-09-22,Its Tesla day! Whether you think Elon Musk is ...,0.4574,0.079,0.000,0.921
2,2020-10-09,"JB Straubel, the Tesla co-founder and former C...",0.0000,0.000,0.000,1.000
3,2020-09-22,Tesla’s battery day is here after months of tw...,0.1779,0.063,0.045,0.892
4,2020-09-23,Tesla CEO Elon Musk revealed Tuesday the Model...,0.4019,0.107,0.000,0.893


# Fetch all News related to Tesla CEO

In [66]:
# Fetch all the news about Elon Musk
elon_musk_headlines = newsapi.get_everything(
    q="elon musk", language="en", sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Elon Musk: {elon_musk_headlines['totalResults']}")

# Show sample article
elon_musk_headlines["articles"][0]



Total articles about Elon Musk: 3034


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Darrell Etherington',
 'title': 'Elon Musk says Tesla will ‘one day’ produce ‘super efficient home HVAC’ with HEPA filtering',
 'description': 'Elon Musk has previously touted the ‘Bioweapon Defense Mode’ boasted by Tesla’s vehicles, which are designed to provide excellent air quality inside the car even in the face of disastrous conditions without, thanks in part to high-efficiency HEPA air filtrati…',
 'url': 'http://techcrunch.com/2020/09/11/elon-musk-says-tesla-will-one-day-produce-super-efficient-home-hvac-with-hepa-filtering/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2020/02/Screen-Shot-2020-02-10-at-12.06.01-PM.jpg?w=546',
 'publishedAt': '2020-09-11T16:00:00Z',
 'content': 'Elon Musk has previously touted the ‘Bioweapon Defense Mode’ boasted by Tesla’s vehicles, which are designed to provide excellent air quality inside the car even in the face of disastrous conditions … [+2200 chars]'}

In [67]:
# Transform the response dictionary to a DataFrame
elon_df = pd.DataFrame.from_dict(elon_musk_headlines["articles"])

elon_df.head()


,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Darrell Etherington,Elon Musk says Tesla will ‘one day’ produce ‘s...,Elon Musk has previously touted the ‘Bioweapon...,http://techcrunch.com/2020/09/11/elon-musk-say...,https://techcrunch.com/wp-content/uploads/2020...,2020-09-11T16:00:00Z,Elon Musk has previously touted the ‘Bioweapon...
1,"{'id': None, 'name': 'Lifehacker.com'}",David Murphy,How to Watch Tesla's 'Battery Day' Today,It’s Tesla day! Whether you think Elon Musk is...,https://lifehacker.com/how-to-watch-teslas-bat...,https://i.kinja-img.com/gawker-media/image/upl...,2020-09-22T19:00:00Z,Its Tesla day! Whether you think Elon Musk is ...
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Kirsten Korosec,Tesla is building a cathode plant and getting ...,"A little more than a year ago, during Tesla’s ...",https://consent.yahoo.com/v2/collectConsent?se...,None,2020-09-23T00:09:43Z,TechCrunch fait partie de Verizon Media. Nos p...
3,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,Elon Musk reveals the Tesla Model S ‘Plaid’ sl...,Tesla CEO Elon Musk revealed Tuesday the Model...,http://techcrunch.com/2020/09/22/elon-musk-rev...,https://techcrunch.com/wp-content/uploads/2020...,2020-09-23T01:48:54Z,Tesla CEO Elon Musk revealed Tuesday the Model...
4,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,SpaceX scales back plans for Starship's first ...,You might want to dial back your expectations ...,https://www.engadget.com/spacex-starship-test-...,https://o.aolcdn.com/images/dims?resize=1200%2...,2020-09-26T18:23:24Z,You might want to dial back your expectations ...


### Creste Elon Musk Sentiment Score Table

In [68]:
# Create the Elan MUSK sentiment scores DataFrame
elon_musk_sentiments = []

for article in elon_musk_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        elon_musk_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
elon_df = pd.DataFrame(elon_musk_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
elon_df = elon_df[cols]

elon_df.head()



,date,text,compound,positive,negative,neutral
0,2020-09-11,Elon Musk has previously touted the ‘Bioweapon...,0.0258,0.132,0.13,0.738
1,2020-09-22,Its Tesla day! Whether you think Elon Musk is ...,0.4574,0.079,0.00,0.921
2,2020-09-23,TechCrunch fait partie de Verizon Media. Nos p...,0.0000,0.000,0.00,1.000
3,2020-09-23,Tesla CEO Elon Musk revealed Tuesday the Model...,0.4019,0.107,0.00,0.893
4,2020-09-26,You might want to dial back your expectations ...,0.2732,0.081,0.00,0.919
